In [9]:
import pandas as pd
from glob import glob
import json, os
from sklearn.model_selection import train_test_split

### DF

In [2]:
new_df = pd.read_csv('D:/압축/ai_data.csv', index_col = 'file_name')
new_df.drop('type',axis=1, inplace=True)

### 결측치 확인

In [3]:
new_df.isnull().sum()

boxcorners    0
ansize        0
class         0
dtype: int64

### 각 컬럼별 고유값 개수

In [4]:
display(new_df['ansize'].value_counts())
display(new_df['class'].value_counts(ascending=True))

[1920, 1440]    786523
[4000, 3000]     29940
[1, 1]            6247
[1920, 1743]       614
[1849, 1920]       523
                 ...  
[1920, 1720]         1
[1920, 1417]         1
[1920, 1004]         1
[1920, 945]          1
[1920, 1653]         1
Name: ansize, Length: 81, dtype: int64

스킨로션          799
선케어          1217
립케어블러셔       1218
향수           1696
팩마스크         2301
여행용가방        2577
베이스메이크업      2610
안경테          2956
우산양산         3453
클렌징필링        3989
에센스앰플        4352
에어쿠션팩트       4396
운동용가방        4705
기타시계         6374
남성지갑         6686
여성지갑         7627
패션시계        11362
남성가방        11687
아동샌들        13104
아동모자        15337
아동외출여행가방    15968
아동운동화       18110
기능화         18135
남성워커부츠      18211
선글라스        18227
남성정장화       18940
아동구두        19325
기타지갑        19573
아동책가방       27026
여성샌들        30605
아쿠아슈즈쪼리     31810
실내화슬리퍼      38236
운동화         38302
패션모자        38938
여성단화        41397
남성캐주얼화      42179
힐펌프스        47285
여성부츠워커      50976
캐쥬얼가방       54022
여성가방        62450
스니커즈슬립온     72519
Name: class, dtype: int64

In [ ]:
### 해상도 1920, 1440만 사용

In [5]:
new_list =[]
for i in range(len(new_df)):
    if new_df.iloc[i]['ansize'] == '[1920, 1440]':
        new_list.append(new_df.iloc[i])
aa = pd.DataFrame(new_list)
zz = []
for i in aa.index:
    zz.append(i.replace('.JPG','.jpg'))
aa.reset_index(inplace=True)
aa['filename'] = zz
aa.drop('index',axis=1,inplace=True)
aa.to_csv('E:/1920_1440.csv', encoding='utf-8')

## Train_set

In [17]:
X = []
# for i in aa['filename']:
#     X.append(i)
for i in glob('D:/ai_set/*.jpg'):
    X.append(i)

train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10


# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
x_train, x_test = train_test_split(X, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test = train_test_split(x_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

for i in x_train:
    with open('C:/Users/HumanForest/Desktop/darknet/build/darknet/x64/data/ai_product/train.txt','a', encoding='utf-8') as f:
            f.write(i+'\n')
            f.close()
for i in x_val:
    with open('C:/Users/HumanForest/Desktop/darknet/build/darknet/x64/data/ai_product/val.txt','a', encoding='utf-8') as f:
            f.write(i+'\n')
            f.close()
for i in x_test:
    with open('C:/Users/HumanForest/Desktop/darknet/build/darknet/x64/data/ai_product/test.txt','a', encoding='utf-8') as f:
            f.write(i+'\n')
            f.close()


### Convert to Json(TF_Version)

In [ ]:
from glob import glob
aa = []
bb = []
with open('D:/img_list.txt',encoding='utf-8') as f:
    for i in f:
        aa.append(i.split('\n')[0])
with open('D:/img_list1.txt',encoding='utf-8') as f:
    for i in f:
        bb.append(i.split('\n')[0])
print(len(aa), len(bb))
change_dict = dict(zip(bb, aa))
from glob import glob 
import os, json, shutil

class_dict = {}
with open('C:/Users/HumanForest/Desktop/darknet/build/darknet/x64/classes.txt', encoding='utf-8') as aa :
    for i,j in enumerate(aa):
        class_dict[j.split('\n')[0]] = i 
aa = []
for i in glob('D:/ai_set/*.txt'):
    aa.append(i.split('\\')[-1].split('.')[0])

for i in glob('D:/압축/**/*.jpg', recursive=True):
    file_name = i.split('\\')[-1]
    try:
        if i.split('\\')[-1].split('.')[0] in aa:
            shutil.copy(i, 'E:/ai_product/{}'.format(file_name))
            with open(i.replace('.jpg','.json'), encoding='utf-8') as json_file:
                json_data = json.load(json_file)
                ann = json_data['regions'][0]
                class_id = str(class_dict[ann['class']])
                x1, y1, x2, y2 = str(ann['boxcorners'][0])+',', str(ann['boxcorners'][1])+',', str(ann['boxcorners'][2])+',', str(ann['boxcorners'][3])+','
                with open('E:/train_set.txt', 'a', encoding='utf-8') as F:
                    F.write('E:/ai_product/{}'.format(file_name) + ' ' + x1 + y1 + x2 + y2 + class_id +'\n')
                    F.close()
    except Exception as ex:
        print(ex)
        print(i)
         
if 'HF020171_0133_0027.jpg' in change_dict.keys(): 
    print(True)




### Convert to Json(darknet_Version) ** Normalization

In [1]:
import shutil
import json
from glob import glob

test_set = []
train_set = []
with open('D:/압축/test_img.txt', encoding='utf-8') as f:
    for i in f:
        test_set.append(i.replace('.jpg','.json').split('\n')[0])
with open('D:/압축/train_img.txt', encoding='utf-8') as f:
    for i in f:
        train_set.append(i.replace('.jpg','.json').split('\n')[0])

class_dict = {}
with open('C:/Users/HumanForest/Desktop/darknet/build/darknet/x64/classes.txt', encoding='utf-8') as aa :
    for i,j in enumerate(aa):
        class_dict[j.split('\n')[0]] = i 

for j_file in glob('D:/압축/**/*.json', recursive = True):    
    try:
        if j_file in test_set or train_set:
            with open(j_file, encoding='utf-8') as json_file:
                jsonData = json.load(json_file)
                img_x, img_y = jsonData['image']['imsize'][0], jsonData['image']['imsize'][1]  # Width, High of Image
                ann = jsonData['regions']

                check_set = set()
                for i in range(len(ann)):
                    if 'boxcorners' in ann[i]:
                        bb = ann[i]['boxcorners']
                        class_id = class_dict[ann[i]['class']]

                        x = round(abs(bb[0] + bb[2]) / 2 / img_x, 6)
                        y = round(abs(bb[1] + bb[3]) / 2 / img_y, 6)
                        w = round(abs(bb[2] - bb[0]) / img_x, 6)
                        h = round(abs(bb[3] - bb[1]) / img_y, 6)
                        content = str(class_id) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) # Yolov4 data set

                        if j_file in check_set:
                            # Append to file files
                            file = open(os.path.join('D:/ai_set', j_file.replace('.json','.txt').split('\\')[-1]), 'a', encoding='utf-8')
                            file.write('\n')
                            file.write(content)
                            file.close()
                        elif j_file not in check_set:
                            check_set.add(j_file)
                            # Write files
                            file = open(os.path.join('D:/ai_set', j_file.replace('.json','.txt').split('\\')[-1]), 'w', encoding='utf-8')
                            file.write(content)
                            file.close()
    except Exception as ex:
        print(ex)
        print(j_file)
        pass

Expecting value: line 1 column 1 (char 0)
D:/압축\folder\05_상품\HF020066_상품_신발_슬리퍼기능화_실내화슬리퍼_지압슬리퍼\HF020066_0104_1462.json
